In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import scipy.stats as stats

def boxplot_male_female():
    file_path = 'Results.csv'  # Specify the file path of Results.csv
    results = pd.read_csv(file_path)  # Read the CSV file into a dataframe
    column_names = [f'{test}_correct_rate' for test in ['spatial', 'memory', 'math', 'ans']]  # Defines rate columns
    results.dropna(subset=column_names, how='any', inplace=True)
    
    ## Now assign the results to male_results_rates and female_results_rates
    male_results_rates = results.loc[results['gender'] == 'M', column_names]
    female_results_rates = results.loc[results['gender'] == 'F', column_names]
    
    fig, ax = plt.subplots()
    positions = range(1, 2*len(column_names), 2)  # Generate positions for each pair of boxplots
    
    # Plotting each pair of boxplots with hatching
    for test, pos in zip(column_names, positions):
        male_box = ax.boxplot(male_results_rates[test], positions=[pos], widths=0.6, patch_artist=True)
        female_box = ax.boxplot(female_results_rates[test], positions=[pos + 0.8], widths=0.6, patch_artist=True)
    
        for box in male_box['boxes']:
            box.set_facecolor('white')  # Neutral color
            box.set_hatch('//')  # Hatching pattern for male
        for box in female_box['boxes']:
            box.set_facecolor('white')  # Neutral color
            box.set_hatch('\\\\')  # Hatching pattern for female
    
        # Customizing the median lines
        for median in male_box['medians']:
            median.set_color('black')
            median.set_linewidth(2)
        for median in female_box['medians']:
            median.set_color('black')
            median.set_linewidth(2)
    
    # Male and Female labels for y-axis
    for pos in positions:
        ax.text(pos, -0.05, 'Male', ha='center', va='top', rotation=60)
        ax.text(pos + 0.8, -0.05, 'Female', ha='center', va='top', rotation=60)
    
    # Test names (below gender labels)
    ax.set_xticks([(pos + pos + 0.8) / 2 for pos in positions])
    ax.set_xticklabels([test.split('_')[0] + ' test' for test in column_names], position=(10, -0.15))
    
    ax.set_ylabel("Rate of Correct Answers")  # y-axis label
    ax.set_title("Comparison of Cognitive Test Scores by Gender", loc='center')  # Title
    
    plt.tight_layout()  # Adjust layout
    plt.show()
    
def bonferroni_male_female():
    ###bonferroni
    test_results = []  # Empty list to hold our test results
    
    # t-tests for each cognitive test
    for test in column_names:
        # Extract scores for males and females for the current test
        male_scores = male_results_rates[test]
        female_scores = female_results_rates[test]
        
        # Check for equal variances
        levene_stat, levene_p = stats.levene(male_scores, female_scores)
        equal_var = levene_p > 0.05
        
        # Perform t-test
        t_test_stat, p_value = stats.ttest_ind(male_scores, female_scores, equal_var=equal_var)
        
        # Bonferroni correction: multiply p-value by the number of tests (but not more than 1)
        p_val_bonferroni = min(p_value * len(column_names), 1)
        
        # Append the results
        test_results.append({
            'Test': test.replace('_correct_rate', '').capitalize(),
            'Mean Score Male': male_scores.mean(),
            'Standard Error Male': stats.sem(male_scores),
            'Mean Score Female': female_scores.mean(),
            'Standard Error Female': stats.sem(female_scores),
            'Difference in Score': male_scores.mean() - female_scores.mean(),
            'P-Value (Bonferroni corrected)': p_val_bonferroni
        })
    
    # Convert the results into a DataFrame
    test_results_df = pd.DataFrame(test_results)
    
    # Reorder DataFrame to match the desired output format
    test_results_df = test_results_df[['Test', 'Mean Score Male', 'Standard Error Male', 
                                       'Mean Score Female', 'Standard Error Female', 
                                       'Difference in Score', 'P-Value (Bonferroni corrected)']]
    
    # Print the DataFrame
    print(test_results_df.to_string(index=False))

| Test    | Mean Score Male | Standard Error Male | Mean Score Female | Standard Error Female | Difference in Score | P-Value (Bonferroni corrected) |
|---------|-----------------|---------------------|-------------------|-----------------------|---------------------|--------------------------------|
| Spatial | 0.700           | 0.073               | 0.609             | 0.059                 | 0.091               | 1.000                          |
| Memory  | 0.385           | 0.042               | 0.457             | 0.041                 | -0.072              | 1.000                          |
| Math    | 0.849           | 0.034               | 0.877             | 0.022                 | -0.028              | 1.000                          |
| Ans     | 0.558           | 0.025               | 0.535             | 0.030                 | 0.023               | 1.000                          |